## 自然言語処理で役に立つGinzaの使い方

In [1]:
# colabでインストール
!pip install ginza==4.0.5

In [23]:
# Ginzaを使うにはspaCyをインポートします。
import spacy
nlp = spacy.load('ja_ginza')

## トークン化の基本

In [24]:
doc = nlp('今日の晩御飯はカレーです。')
for token in doc:
  print(token)

今日
の
晩御飯
は
カレー
です
。


## **形態素解析**

### 1.トークン化の３パターン


1.   最小限の単位（Aパターン）
2.   上記+接辞、一部の複合動詞（Bパターン）
3.   固有名詞、慣用句など（Cパターン）



In [25]:
#Aパターンの例
#set_split_modeで切り替え
import ginza
ginza.set_split_mode(nlp,'A')
doc = nlp('私は東京都の裁判官です。')
for token in doc:
  print(token)

私
は
東京
都
の
裁判
官
です
。


In [26]:
#Bパターンの例
ginza.set_split_mode(nlp,'B')
doc = nlp('私は東京都の裁判官です。')
for token in doc:
  print(token)

私
は
東京都
の
裁判官
です
。


In [27]:
#Cパターンの例
ginza.set_split_mode(nlp,'C')
doc = nlp('私は東京都住まいの国家公務員です。')
for token in doc:
  print(token)

私
は
東京都
住まい
の
国家公務員
です
。


### 2.品詞のタグ

In [28]:
ginza.set_split_mode(nlp,'A')
doc = nlp('私は北海道に行きます。')
for token in doc:
  print(token.text + ',' + token.tag_ + ',' + token.pos_)

私,代名詞,PRON
は,助詞-係助詞,ADP
北海道,名詞-固有名詞-地名-一般,PROPN
に,助詞-格助詞,ADP
行き,動詞-非自立可能,VERB
ます,助動詞,AUX
。,補助記号-句点,PUNCT


### 3.Lemmatization（レンマ化）

**レンマ化→トークンを見出し語に変換**<br>
例：<br>
行こう　→　行く<br>
didn't →　did not


In [29]:
doc = nlp('私は北海道に行きます。')
for token in doc:
  print(token.text + ',' + token.lemma_)

私,私
は,は
北海道,北海道
に,に
行き,行く
ます,ます
。,。


### 4.文境界解析

単語だけではなく、文章も分割できる。

In [30]:
doc = nlp('明日はみんなで映画館に行きます。今から楽しみです。')
for sentence in doc.sents:
  print(sentence)

明日はみんなで映画館に行きます。
今から楽しみです。


さらに文章からトークンを抜き出すことができます。

In [31]:
doc = nlp('明日はみんなで映画館に行きます。今から楽しみです。')
for sentence in doc.sents:
  for token in sentence:
    print(token)

明日
は
みんな
で
映画
館
に
行き
ます
。
今
から
楽し
み
です
。


### 5.文節区切り

In [32]:
doc = nlp('明日はみんなで映画館に行きます。今から楽しみです。')
for sentence in doc.sents:
  for span in ginza.bunsetu_spans(sentence):
    print(span)

明日は
みんなで
映画館に
行きます。
今から
楽しみです。


※文節区切りからでもトークンを取り出せる

In [33]:
doc = nlp('明日はみんなで映画館に行きます。今から楽しみです。')
for sentence in doc.sents:
  for span in ginza.bunsetu_spans(sentence):
    for token in span:
      print(token)

明日
は
みんな
で
映画
館
に
行き
ます
。
今
から
楽し
み
です
。


## 係り受け解析

トークン同士の関係性を解析する

In [34]:
# 結果を可視化するための関数
from spacy import displacy

In [35]:
doc = nlp('池袋に一緒にラーメンを食べに行きましょう。')

In [36]:
for token in doc:
  print(token.text + ',' + token.head.text + ',' + token.dep_) # headは親トークン,depはタグ

# グラフで係り受け解析の結果を可視化する
displacy.render(doc,style='dep',jupyter=True)

池袋,行き,obl
に,池袋,case
一緒,行き,obl
に,一緒,case
ラーメン,食べ,obj
を,ラーメン,case
食べ,行き,advcl
に,食べ,case
行き,行き,ROOT
ましょう,行き,aux
。,行き,punct


## 固体表現抽出

固有名詞などを判別し、抽出できる。

In [37]:
doc = nlp('今度、佐藤さんと国立新美術館に行きます。')

In [41]:
for entity in doc.ents:
  print(entity.text + ', ' + entity.label_ + ', ' + str(entity.start_char) + ',' + str(entity.end_char))
displacy.render(doc,style='ent',jupyter=True)

佐藤, Person, 3,5
さん, Title_Other, 5,7
国立新美術館, Museum, 8,14


佐藤という固有名詞と国立新美術館というオブジェクト名が抽出されています。